# Agenticデザインパターン実装ガイド

このコードサンプルでは、主要なAgenticデザインパターンを[Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel)を使用して実装します。

## 📚 このノートブックで学習できること

### 🎯 主要な学習目標
- **ReActパターン**の理解と実装
- **プランニングパターン**の活用方法
- **マルチエージェントパターン**の協調設計
- **Reflexion(反省)パターン**による自己改善

### 🛠️ 実装するデザインパターン
- **ReAct**: Reasoning（推論）+ Acting（行動）サイクル
- **Planning**: 段階的なタスク分解と実行
- **Multi-Agent**: 複数エージェントの役割分担
- **Reflexion**: 失敗からの学習と改善

### 💡 実用的なスキル
- パターン別の適用場面判断
- 複雑なタスクの効率的な処理
- エラー回復と自己修正メカニズム
- システムアーキテクチャの設計思想

## 必要なパッケージのインストール

In [1]:
%pip install semantic-kernel openai python-dotenv --quiet --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 必要なPythonパッケージのインポート

In [2]:
import os
import asyncio
import json
from typing import Annotated, Any, Dict, List, Optional
from datetime import datetime
from dataclasses import dataclass
from enum import Enum

from openai import AsyncOpenAI
from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatMessageContent

## 1. ReAct（Reasoning + Acting）パターンの実装

ReActパターンは、推論と行動を交互に行うことで複雑な問題を解決します。

In [ ]:
class ReActToolkit:
    """ReActパターン用のツールキット"""
    
    def __init__(self):
        self.knowledge_base = {
            "東京": {
                "population": "約1400万人",
                "famous_spots": ["東京タワー", "スカイツリー", "浅草", "渋谷"],
                "transportation": "JR、地下鉄、バス",
                "weather": "温帯気候"
            },
            "京都": {
                "population": "約147万人", 
                "famous_spots": ["清水寺", "金閣寺", "伏見稲荷", "嵐山"],
                "transportation": "JR、京阪、阪急",
                "weather": "内陸性気候"
            }
        }
        
        self.calculation_history = []

    @kernel_function(description="都市に関する基本情報を取得")
    def get_city_info(
        self, 
        city: Annotated[str, "情報を取得する都市名"]
    ) -> Annotated[str, "都市の基本情報"]:
        if city in self.knowledge_base:
            info = self.knowledge_base[city]
            return f"""
{city}の基本情報:
- 人口: {info['population']}
- 有名スポット: {', '.join(info['famous_spots'])}
- 交通: {info['transportation']}
- 気候: {info['weather']}
"""
        return f"{city}の情報は見つかりませんでした。"

    @kernel_function(description="数値計算を実行")
    def calculate(
        self, 
        expression: Annotated[str, "計算式（例: 100*3+50）"]
    ) -> Annotated[str, "計算結果"]:
        try:
            # 安全な計算のため、evalの代わりに限定的な実装
            # 実際のアプリケーションではより厳密な計算ライブラリを使用
            allowed_chars = set('0123456789+-*/(). ')
            if all(c in allowed_chars for c in expression):
                result = eval(expression)
                self.calculation_history.append({
                    "expression": expression,
                    "result": result,
                    "timestamp": datetime.now()
                })
                return f"計算結果: {expression} = {result}"
            else:
                return "無効な計算式です。数字と基本的な演算子のみ使用できます。"
        except Exception as e:
            return f"計算エラー: {str(e)}"

    @kernel_function(description="これまでの計算履歴を取得")
    def get_calculation_history(self) -> Annotated[str, "計算履歴"]:
        if not self.calculation_history:
            return "計算履歴はありません。"
        
        history = []
        for calc in self.calculation_history[-5:]:  # 最新5件
            history.append(f"{calc['expression']} = {calc['result']}")
        
        return f"最新の計算履歴:\n" + "\n".join(history)

    @kernel_function(description="推論プロセスを記録")
    def record_reasoning(
        self, 
        step: Annotated[str, "推論ステップの説明"]
    ) -> Annotated[str, "記録完了メッセージ"]:
        print(f"🧠 推論ステップ: {step}")
        return f"推論ステップを記録しました: {step}"

    @kernel_function(description="行動の実行結果を記録")  
    def record_action(
        self, 
        action: Annotated[str, "実行した行動の説明"]
    ) -> Annotated[str, "行動記録完了メッセージ"]:
        print(f"⚡ 実行した行動: {action}")
        return f"行動を記録しました: {action}"

    @kernel_function(description="観察結果を記録し、次のステップを検討")
    def record_observation(
        self,
        observation: Annotated[str, "観察した内容や得られた情報"]
    ) -> Annotated[str, "観察記録完了メッセージ"]:
        print(f"👁️ 観察結果: {observation}")
        return f"観察を記録しました: {observation}"

    @kernel_function(description="2つの都市の特定の項目を比較")
    def compare_cities(
        self,
        city1: Annotated[str, "比較する都市名1"],
        city2: Annotated[str, "比較する都市名2"], 
        aspect: Annotated[str, "比較する項目（population, spots, transportation, weather）"]
    ) -> Annotated[str, "比較結果"]:
        if city1 not in self.knowledge_base or city2 not in self.knowledge_base:
            return f"比較できません。{city1}または{city2}の情報がありません。"
        
        info1 = self.knowledge_base[city1]
        info2 = self.knowledge_base[city2]
        
        if aspect == "population":
            return f"""
都市人口の比較:
- {city1}: {info1['population']}
- {city2}: {info2['population']}
"""
        elif aspect == "spots":
            return f"""
有名スポットの比較:
- {city1}: {', '.join(info1['famous_spots'])}
- {city2}: {', '.join(info2['famous_spots'])}
"""
        elif aspect == "transportation":
            return f"""
交通システムの比較:
- {city1}: {info1['transportation']}
- {city2}: {info2['transportation']}
"""
        elif aspect == "weather":
            return f"""
気候の比較:
- {city1}: {info1['weather']}
- {city2}: {info2['weather']}
"""
        else:
            return f"比較項目 '{aspect}' は対応していません。population, spots, transportation, weatherから選択してください。"

    @kernel_function(description="人口データから数値のみを抽出")
    def extract_population_number(
        self,
        population_text: Annotated[str, "人口を表すテキスト（例：約1400万人）"]
    ) -> Annotated[str, "抽出された数値情報"]:
        import re
        # 数値を抽出するパターン
        numbers = re.findall(r'(\d+)', population_text)
        if numbers:
            # 最初に見つかった数値を使用
            base_number = int(numbers[0])
            if "万" in population_text:
                actual_number = base_number * 10000
                return f"抽出された人口数値: {actual_number}人 (元のテキスト: {population_text})"
            else:
                return f"抽出された人口数値: {base_number}人 (元のテキスト: {population_text})"
        else:
            return f"数値を抽出できませんでした: {population_text}"

In [ ]:
# ReActエージェントの作成と実行

async def demonstrate_react_pattern():
    """ReActパターンのデモンストレーション（段階的出力でNotebook制限を回避）"""
    
    # 環境変数の読み込み
    load_dotenv()
    
    # OpenAIクライアントの設定（GitHub Models経由）
    client = AsyncOpenAI(
        api_key=os.environ.get("GITHUB_TOKEN"), 
        base_url="https://models.inference.ai.azure.com/",
    )
    
    chat_completion_service = OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini",
        async_client=client,
    )
    
    # ReActツールキットの初期化
    react_toolkit = ReActToolkit()
    
    # ReActエージェントの作成（学習用の指示は従来どおり維持）
    react_agent = ChatCompletionAgent(
        service=chat_completion_service,
        plugins=[react_toolkit],
        name="ReActAgent",
        instructions="""
        あなたはReAct（Reasoning + Acting）パターンを使用するAIエージェントです。
        
        **必須のReActサイクル実行手順:**
        問題を解決するため、以下の手順を厳密に守ってください：

        各ステップで必ず以下の3つを順番に実行：
        1. **Reasoning（推論）**: record_reasoningで「なぜこのステップが必要か」を記録
        2. **Acting（行動）**: 具体的なツールを使用し、record_actionで「何を実行したか」を記録  
        3. **Observation（観察）**: record_observationで「結果をどう解釈し、次に何をするか」を記録

        **与えられた問題の5つのステップすべてを実行してください:**
        ステップ1: 両都市の基本情報取得 → get_city_info を使用
        ステップ2: 人口比較と数値計算 → compare_cities, extract_population_number, calculate を使用
        ステップ3: 観光地数の比較 → compare_cities を使用
        ステップ4: 交通システム分析 → compare_cities を使用  
        ステップ5: 総合的観光適性判断 → これまでの結果を統合して最終判断

        **重要**: 各ステップで推論→行動→観察を必ず3回実行してから次のステップに進んでください。
        すべてのステップが完了するまで継続してください。
        """,
    )
    
    print("🔄 ReActパターンデモンストレーション（段階的出力）")
    print("=" * 60)
    
    # フェーズ（段階）ごとに小分けに実行してNotebookの出力制限を回避
    phases = [
        {
            "name": "ステップ1: 基本情報収集",
            "query": "東京と京都の基本情報を取得してください。record_reasoning, record_action, record_observationを使って詳細にプロセスを記録してください。"
        },
        {
            "name": "ステップ2: 人口比較と数値計算",
            "query": "東京と京都の人口を数値で比較し、差や倍率を計算してください。extract_population_number と calculate を使い、各段階でrecord_reasoning/record_action/record_observationを記録してください。"
        },
        {
            "name": "ステップ3: 観光地の比較",
            "query": "両都市の観光地（数・代表例）を比較してください。compare_cities を使い、過程を詳細に記録してください。"
        },
        {
            "name": "ステップ4: 交通システムの分析",
            "query": "両都市の交通システムの違いを、観光客の利便性の観点から分析してください。compare_cities を使い、詳細に記録してください。"
        },
        {
            "name": "ステップ5: 総合判断",
            "query": "これまでの分析結果を統合し、どちらが観光により適しているかを最終判断してください。根拠も併記してください。"
        }
    ]
    
    total_chars = 0
    
    for i, phase in enumerate(phases, 1):
        print(f"\n{'-'*20} {phase['name']} {'-'*20}")
        print(f"📝 クエリ: {phase['query']}\n")
        
        thread = None
        response_text = ""
        
        # 各フェーズを独立してストリーミング実行
        async for response in react_agent.invoke_stream(messages=phase['query'], thread=thread):
            chunk = str(response)
            print(chunk, end="", flush=True)
            response_text += chunk
            thread = response.thread
        
        phase_len = len(response_text)
        total_chars += phase_len
        print(f"\n\n✅ {phase['name']} 完了 | 📊 出力文字数: {phase_len} 文字")
        
        # スレッドを都度破棄して履歴膨張を防止（出力制限回避に有効）
        if thread:
            await thread.delete()
        
        # 次フェーズまで短い待機
        if i < len(phases):
            await asyncio.sleep(1)
    
    print("\n" + "=" * 60)
    print("🎉 全ステップ完了！段階的にReActサイクルを実行しました")
    print(f"📈 総出力文字数: {total_chars} 文字")

# 実行
await demonstrate_react_pattern()

# プランニングパターンのデモンストレーション
次のセルでは、プランニングパターンのデモンストレーションを実行します。
複雑なタスクを段階に分解し、create_plan → get_next_step → complete_step の流れを通しで示します。

In [ ]:
# プランニング用ツールクラス（PlanExecutor）

class PlanExecutor:
    """タスクをステップに分解して進捗管理するプラン実行ツール"""

    def __init__(self):
        self.plan: Dict[str, Any] | None = None
        self.completed: set[int] = set()
        self.results: Dict[int, str] = {}

    @kernel_function(description="タスク説明から実行プラン（ステップ列）を作成")
    def create_plan(
        self,
        task: Annotated[str, "計画対象のタスク説明"]
    ) -> Annotated[str, "作成したプラン(JSON)"]:
        # シンプルなテンプレートでステップを生成（デモ用）
        steps = [
            {"id": 1, "description": "予算の設定と配分を決める", "dependencies": []},
            {"id": 2, "description": "宿泊先の候補を調査・予約", "dependencies": [1]},
            {"id": 3, "description": "観光スポットの選定と日程割り当て", "dependencies": [1]},
            {"id": 4, "description": "交通手段と移動計画を決定", "dependencies": [1]},
            {"id": 5, "description": "最終行程表の作成と確認事項のリスト化", "dependencies": [2, 3, 4]},
        ]
        self.plan = {"task": task, "steps": steps}
        self.completed = set()
        self.results = {}
        print("🗂️ プランを作成しました（ステップ数: {}）".format(len(steps)))
        return json.dumps({"steps": steps}, ensure_ascii=False, indent=2)

    @kernel_function(description="次に実行可能なステップを取得")
    def get_next_step(self) -> Annotated[str, "次のステップ(JSON) または 完了メッセージ"]:
        if not self.plan:
            msg = "プランが作成されていません。まず create_plan を呼び出してください。"
            print(f"⚠️ {msg}")
            return msg
        for step in self.plan["steps"]:
            if step["id"] not in self.completed and all(dep in self.completed for dep in step.get("dependencies", [])):
                print(f"➡️ 次に実行するステップ: {step['id']} - {step['description']}")
                return json.dumps(step, ensure_ascii=False)
        print("🎉 すべてのステップが完了しました")
        return "すべてのステップが完了しました"

    @kernel_function(description="ステップを完了として記録")
    def complete_step(
        self,
        step_id: Annotated[int, "完了するステップID"],
        note: Annotated[str, "実行結果やメモ"]
    ) -> Annotated[str, "完了結果サマリ"]:
        if not self.plan:
            msg = "プランが作成されていません。"
            print(f"⚠️ {msg}")
            return msg
        sid = int(step_id)
        # ステップの存在確認
        valid_ids = {s["id"] for s in self.plan["steps"]}
        if sid not in valid_ids:
            msg = f"ステップ {sid} はプランに存在しません"
            print(f"⚠️ {msg}")
            return msg
        self.completed.add(sid)
        self.results[sid] = note
        remaining = [s["id"] for s in self.plan["steps"] if s["id"] not in self.completed]
        print(f"✅ ステップ{sid}を完了。残り: {remaining}")
        return f"ステップ{sid}を完了しました。残りステップ: {remaining}"

In [ ]:
async def demonstrate_planning_pattern():
    """プランニングパターンのデモンストレーション（読みやすい要約出力）"""
    
    # 環境変数の読み込み
    load_dotenv()
    
    # OpenAIクライアントの設定（本デモではツール実行の可視化が主目的のため、説明用途のみ）
    client = AsyncOpenAI(
        api_key=os.environ.get("GITHUB_TOKEN"), 
        base_url="https://models.inference.ai.azure.com/",
    )
    chat_completion_service = OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini",
        async_client=client,
    )
    
    # プランニングツールの初期化
    plan_executor = PlanExecutor()
    
    print("📋 プランニングパターン（全ステップを読みやすく可視化）")
    print("=" * 60)
    
    # タスク定義
    task = (
        "週末の家族旅行（2泊3日）を計画してください。予算、宿泊、観光、交通すべて含めて段階的に計画を立ててください。"
    )
    print(f"📝 タスク: {task}\n")
    
    # プラン作成（ツールを直接呼び出し）
    print("— フェーズ0: プラン作成 —")
    plan_json_str = plan_executor.create_plan(task)
    try:
        plan_steps = json.loads(plan_json_str).get("steps", [])
    except Exception:
        plan_steps = []
    
    # プラン概要（箇条書き）
    print("🗂️ プラン概要（ステップ一覧）")
    for s in plan_steps:
        deps = ",".join(str(d) for d in s.get("dependencies", [])) or "なし"
        print(f"- Step {s['id']}: {s['description']}  |  依存: {deps}")
    print("")
    
    # 実行フェーズ（人が読みやすい形で理由/ツール/結果のみ表示）
    print("— フェーズ1..N: ステップ実行 —")
    max_loops = 50
    for _ in range(max_loops):
        next_step_str = plan_executor.get_next_step()
        if isinstance(next_step_str, str) and "すべてのステップが完了しました" in next_step_str:
            print("🎉 すべてのステップが完了しました。\n")
            break
        
        # JSON解析
        try:
            step_obj = json.loads(next_step_str)
            step_id = int(step_obj.get("id"))
            step_desc = step_obj.get("description", "")
        except Exception:
            print("⚠️ 次ステップの解析に失敗しました。処理を終了します。")
            break
        
        # ステップブロック（読みやすい整形）
        print(f"┌─ Step {step_id}: {step_desc}")
        print("├ 理由: 依存関係が満たされたため次の実行ステップとして選択")
        print(f"├ ツール: complete_step(step_id={step_id}, note='学習デモ: {step_desc} を完了')")
        result = plan_executor.complete_step(step_id=step_id, note=f"学習デモ: {step_desc} を完了")
        print(f"└ 結果: {result}\n")
        
        await asyncio.sleep(0.1)
    
    # サマリ
    print("— サマリ —")
    print(f"完了したステップ: {sorted(list(plan_executor.completed))}")
    if plan_executor.results:
        print("実行メモ:")
        for sid in sorted(plan_executor.results.keys()):
            print(f"- Step {sid}: {plan_executor.results[sid]}")
    
    print("\n✅ プランニングパターン（要約表示）完了")

await demonstrate_planning_pattern()

## 3. マルチエージェントパターンの実装

複数のエージェントが協調して問題解決を行うパターンです。

次の2セルでは通信ハブ実装とデモ実行を通じて、エージェント登録・メッセージ送受信・各フェーズの作業完了ログが表示されます。これにより、役割分担と連携の流れを実行結果で確認できます。

In [3]:
class CommunicationHub:
    """エージェント間のコミュニケーションを管理"""
    
    def __init__(self):
        self.messages: List[Dict[str, Any]] = []
        self.active_agents: List[str] = []

    @kernel_function(description="他のエージェントにメッセージを送信")
    def send_message(
        self,
        from_agent: Annotated[str, "送信者エージェント名"],
        to_agent: Annotated[str, "受信者エージェント名（'Analyst' など、カンマ区切り可。'all'/'*'/'broadcast' でブロードキャスト）"],
        message: Annotated[str, "メッセージ内容"]
    ) -> Annotated[str, "送信結果"]:
        # 複数宛先 or ブロードキャスト対応
        targets: List[str] = []
        ta = (to_agent or "").strip()
        if "," in ta:
            targets = [t.strip() for t in ta.split(",") if t.strip()]
        elif ta.lower() in ("all", "*", "broadcast"):
            # 登録済みエージェントに配信（送信者自身は除外）。未登録なら 'all' で1件送信
            if self.active_agents:
                targets = [a for a in self.active_agents if a != from_agent]
            else:
                targets = ["all"]
        else:
            targets = [ta]
        
        sent_count = 0
        for tgt in targets:
            msg = {
                "id": len(self.messages) + 1,
                "from": from_agent,
                "to": tgt,
                "content": message,
                "timestamp": datetime.now().isoformat()
            }
            self.messages.append(msg)
            print(f"📨 {from_agent} → {tgt}: {message}")
            sent_count += 1
        
        return f"{sent_count}件のメッセージを送信しました"

    @kernel_function(description="自分宛のメッセージを取得")
    def get_messages(
        self,
        agent_name: Annotated[str, "エージェント名"]
    ) -> Annotated[str, "受信メッセージ"]:
        # 自分宛て + ブロードキャスト（all/*/broadcast）を受信対象に含める
        agent_messages = [
            msg for msg in self.messages 
            if msg["to"] == agent_name or str(msg["to"]).lower() in ("all", "*", "broadcast")
        ]
        
        if not agent_messages:
            return f"{agent_name} 宛のメッセージはありません"
        
        result = f"{agent_name} の受信メッセージ:\n"
        for msg in agent_messages[-3:]:  # 最新3件
            result += f"- {msg['from']}: {msg['content']}\n"
        
        return result

    @kernel_function(description="エージェントの稼働状況を登録")
    def register_agent(
        self,
        agent_name: Annotated[str, "エージェント名"]
    ) -> Annotated[str, "登録結果"]:
        if agent_name not in self.active_agents:
            self.active_agents.append(agent_name)
        return f"エージェント {agent_name} を登録しました"

    @kernel_function(description="稼働中のエージェント一覧を取得")
    def list_active_agents(self) -> Annotated[str, "稼働中エージェント一覧"]:
        if not self.active_agents:
            return "稼働中のエージェントはいません"
        return f"稼働中のエージェント: {', '.join(self.active_agents)}"

In [4]:
# マルチエージェントシステムのデモンストレーション

async def demonstrate_multi_agent_pattern():
    """マルチエージェントパターンのデモンストレーション（確実なメッセージ授受版）"""
    
    # 環境変数の読み込み
    load_dotenv()
    
    # OpenAIクライアントの設定
    client = AsyncOpenAI(
        api_key=os.environ.get("GITHUB_TOKEN"), 
        base_url="https://models.inference.ai.azure.com/",
    )
    
    chat_completion_service = OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini",
        async_client=client,
    )
    
    # 通信ハブの初期化
    comm_hub = CommunicationHub()
    
    # 研究者エージェントの作成
    researcher_agent = ChatCompletionAgent(
        service=chat_completion_service,
        plugins=[comm_hub],
        name="Researcher",
        instructions="""
        あなたは情報収集・分析を専門とする研究者エージェントです。
        調査結果は簡潔に箇条書きし、必要なら CommunicationHub の send_message で共有してください。
        """,
    )
    
    # 分析者エージェントの作成
    analyst_agent = ChatCompletionAgent(
        service=chat_completion_service,
        plugins=[comm_hub],
        name="Analyst",
        instructions="""
        あなたはデータ分析・解釈を専門とする分析者エージェントです。
        受け取った情報を要約→パターン抽出→示唆の順で分析し、簡潔にまとめてください。
        必要なら CommunicationHub の send_message で Planner に共有してください。
        """,
    )
    
    # 企画者エージェントの作成
    planner_agent = ChatCompletionAgent(
        service=chat_completion_service,
        plugins=[comm_hub],
        name="Planner",
        instructions="""
        あなたは戦略企画を専門とする企画者エージェントです。
        受け取った洞察から、実行可能な施策を優先度・インパクト・実現性の観点で 3〜5 件に整理し、最終案を提示してください。
        """,
    )
    
    print("👥 マルチエージェントパターンデモンストレーション")
    print("=" * 50)
    
    # テスト用のトピック
    topic = "リモートワーク導入による企業の生産性向上について調査・分析し、具体的な施策を提案してください"
    print(f"📝 プロジェクト: {topic}\n")
    
    # 事前に全エージェントを登録（ブロードキャスト宛先解決のため）
    print(comm_hub.register_agent("Researcher"))
    print(comm_hub.register_agent("Analyst"))
    print(comm_hub.register_agent("Planner"))
    
    latest_broadcast = ""
    analyst_to_planner = ""
    
    # フェーズ1: Researcher
    print(f"\n🔄 フェーズ 1: Researcher の作業開始")
    print("-" * 40)
    research_task = (
        f"このトピックを調査し、主要ファクト・統計・論点を箇条書きで簡潔にまとめてください。トピック: {topic}"
    )
    print(f"📋 タスク: {research_task}\n")
    collected: list[str] = []
    thread = None
    async for response in researcher_agent.invoke_stream(messages=research_task, thread=thread):
        s = str(response)
        if s:
            collected.append(s)
        thread = getattr(response, "thread", thread)
    research_text = "".join(collected).strip()
    # 軽い整形
    research_text = research_text.strip()
    if len(research_text) > 1800:
        research_text = research_text[:1800] + "\n…（省略）"
    print(research_text + ("\n" if not research_text.endswith("\n") else ""))
    
    # 研究結果を CommunicationHub にブロードキャスト（確実に Analyst/Planner が受信）
    latest_broadcast = research_text or "（研究結果の要点）"
    print("\n📡 研究結果を broadcast 送信します…")
    print(comm_hub.send_message(from_agent="Researcher", to_agent="all", message=latest_broadcast))
    
    print("\n✅ Researcher の作業完了")
    if thread:
        await thread.delete()
    await asyncio.sleep(0.5)
    
    # フェーズ2: Analyst（受信した broadcast を明示的にタスクへ埋め込み）
    print(f"\n🔄 フェーズ 2: Analyst の作業開始")
    print("-" * 40)
    inbox_preview = comm_hub.get_messages(agent_name="Analyst")
    analyst_task = (
        "以下の受信メッセージ（研究結果の要点）を要約→パターン抽出→示唆の順で分析し、"
        "結論を 5〜10 行で簡潔にまとめてください。\n\n" + inbox_preview
    )
    print(f"📋 タスク: 研究者からの受信内容を分析\n")
    collected = []
    thread = None
    async for response in analyst_agent.invoke_stream(messages=analyst_task, thread=thread):
        s = str(response)
        if s:
            collected.append(s)
        thread = getattr(response, "thread", thread)
    analyst_text = "".join(collected).strip()
    if len(analyst_text) > 2000:
        analyst_text = analyst_text[:2000] + "\n…（省略）"
    if analyst_text:
        print(analyst_text + ("\n" if not analyst_text.endswith("\n") else ""))
    else:
        print("（分析結果の本文が生成されませんでした）")
    
    # Analyst → Planner に確実に送信
    analyst_to_planner = analyst_text or "（分析要約）"
    print("\n📨 Analyst → Planner へ分析要約を送信します…")
    print(comm_hub.send_message(from_agent="Analyst", to_agent="Planner", message=analyst_to_planner))
    
    print("\n✅ Analyst の作業完了")
    if thread:
        await thread.delete()
    await asyncio.sleep(0.5)
    
    # フェーズ3: Planner（Analyst からの要約を明示的にタスクへ埋め込み）
    print(f"\n🔄 フェーズ 3: Planner の作業開始")
    print("-" * 40)
    planner_inbox = comm_hub.get_messages(agent_name="Planner")
    planner_task = (
        "以下の受信メッセージ（分析要約）を基に、実行施策 3〜5 件を、優先度・工数・インパクトの観点で整理して提案してください。\n\n"
        + planner_inbox
    )
    print(f"📋 タスク: 分析結果に基づく施策の提示\n")
    collected = []
    thread = None
    async for response in planner_agent.invoke_stream(messages=planner_task, thread=thread):
        s = str(response)
        if s:
            collected.append(s)
        thread = getattr(response, "thread", thread)
    planner_text = "".join(collected).strip()
    if len(planner_text) > 2200:
        planner_text = planner_text[:2200] + "\n…（省略）"
    if planner_text:
        print(planner_text + ("\n" if not planner_text.endswith("\n") else ""))
    else:
        print("（施策案の本文が生成されませんでした）")
    
    print("\n✅ Planner の作業完了")
    if thread:
        await thread.delete()
    
    print("\n" + "=" * 50)
    print("✅ マルチエージェントプロジェクト完了")

await demonstrate_multi_agent_pattern()

👥 マルチエージェントパターンデモンストレーション
📝 プロジェクト: リモートワーク導入による企業の生産性向上について調査・分析し、具体的な施策を提案してください

エージェント Researcher を登録しました
エージェント Analyst を登録しました
エージェント Planner を登録しました

🔄 フェーズ 1: Researcher の作業開始
----------------------------------------
📋 タスク: このトピックを調査し、主要ファクト・統計・論点を箇条書きで簡潔にまとめてください。トピック: リモートワーク導入による企業の生産性向上について調査・分析し、具体的な施策を提案してください

リモートワーク導入による企業の生産性向上に関する調査結果を以下のようにまとめました。

### 主要ファクト・統計
- **生産性の向上**: 多くの研究がリモートワークが生産性を向上させると報告しており、調査によると約77%の企業が生産性が向上したと回答。
- **従業員満足度**: 約70%の従業員がリモートワークを望んでおり、これは雇用者の離職率を減少させる要因。
- **コスト削減**: オフィススペースの削減により、企業は年間約20%のコストを削減できる可能性。
- **移動時間の削減**: 通勤時間が削減されることで、従業員はその時間を仕事や自己開発に利用可能。
- **技術の進展**: リモートワークのためのツール（Zoom, Slackなど）の普及が、円滑なコミュニケーションをサポート。

### 論点
- **チームコラボレーションの課題**: リモート環境でのチームワークやコミュニケーションに課題が存在する。
- **仕事と私生活の境界**: 在宅勤務により、仕事と私生活の境目が曖昧になることへの懸念。
- **従業員の健康**: 長時間の画面前作業や孤立感が心身に与える影響。

### 具体的な施策提案
1. **適切なコミュニケーションツールの導入**: チームメンバーがスムーズにコミュニケーションできるためのツールを提供。
2. **定期的なミーティング**: オンラインでの定期的なチームミーティングを設け、進行状況や問題点を共有。
3. **効果的なワークライフバランスの推進**: フレックスタイ

## 4. Reflexion（反省）パターンの実装

失敗から学習し、自己改善を行うパターンです。
次の2セルでは、(1) ReflexionEngine（試行記録・反省・成功要因・知識取得・改善提案のツール群）を定義し、(2) そのデモを実行して継続的改善サイクルの出力を確認します。

In [5]:
class ReflexionEngine:
    """反省と学習を管理するエンジン"""
    
    def __init__(self):
        self.attempt_history: List[Dict[str, Any]] = []
        self.learned_lessons: List[str] = []
        self.success_patterns: List[str] = []

    @kernel_function(description="新しい試行を記録")
    def record_attempt(
        self,
        attempt_id: Annotated[str, "試行のID"],
        approach: Annotated[str, "使用したアプローチの説明"],
        result: Annotated[str, "結果の説明"],
        success: Annotated[bool, "成功したかどうか"]
    ) -> Annotated[str, "記録完了メッセージ"]:
        attempt = {
            "id": attempt_id,
            "approach": approach,
            "result": result,
            "success": success,
            "timestamp": datetime.now().isoformat()
        }
        
        self.attempt_history.append(attempt)
        
        status = "成功" if success else "失敗"
        print(f"📝 試行記録: {attempt_id} - {status}")
        print(f"   アプローチ: {approach}")
        print(f"   結果: {result}")
        
        return f"試行 {attempt_id} を記録しました ({status})"

    @kernel_function(description="失敗から教訓を抽出")
    def reflect_on_failure(
        self,
        attempt_id: Annotated[str, "失敗した試行のID"],
        analysis: Annotated[str, "失敗の原因分析"]
    ) -> Annotated[str, "反省結果"]:
        # 対象の試行を見つける
        attempt = None
        for a in self.attempt_history:
            if a["id"] == attempt_id:
                attempt = a
                break
        
        if not attempt or attempt["success"]:
            return f"試行 {attempt_id} は失敗していないか、見つかりません"
        
        # 教訓を抽出
        lesson = f"失敗パターン: {attempt['approach']} → 原因: {analysis}"
        self.learned_lessons.append(lesson)
        
        print(f"🧠 教訓を学習: {lesson}")
        return f"試行 {attempt_id} から教訓を学習しました"

    @kernel_function(description="成功パターンを記録")
    def record_success_pattern(
        self,
        attempt_id: Annotated[str, "成功した試行のID"],
        key_factors: Annotated[str, "成功の要因分析"]
    ) -> Annotated[str, "成功パターン記録結果"]:
        # 対象の試行を見つける
        attempt = None
        for a in self.attempt_history:
            if a["id"] == attempt_id:
                attempt = a
                break
        
        if not attempt or not attempt["success"]:
            return f"試行 {attempt_id} は成功していないか、見つかりません"
        
        # 成功パターンを記録
        pattern = f"成功パターン: {attempt['approach']} → 要因: {key_factors}"
        self.success_patterns.append(pattern)
        
        print(f"🌟 成功パターンを記録: {pattern}")
        return f"試行 {attempt_id} の成功パターンを記録しました"

    @kernel_function(description="学習した知識を取得")
    def get_learned_knowledge(self) -> Annotated[str, "学習した知識"]:
        if not self.learned_lessons and not self.success_patterns:
            return "まだ学習した知識はありません"
        
        knowledge = "🧠 学習した知識:\n\n"
        
        if self.learned_lessons:
            knowledge += "📚 教訓 (失敗から学んだこと):\n"
            for i, lesson in enumerate(self.learned_lessons, 1):
                knowledge += f"{i}. {lesson}\n"
            knowledge += "\n"
        
        if self.success_patterns:
            knowledge += "🌟 成功パターン:\n"
            for i, pattern in enumerate(self.success_patterns, 1):
                knowledge += f"{i}. {pattern}\n"
        
        return knowledge

    @kernel_function(description="改善された新しいアプローチを提案")
    def suggest_improved_approach(
        self,
        current_problem: Annotated[str, "現在の問題の説明"]
    ) -> Annotated[str, "改善提案"]:
        if not self.learned_lessons and not self.success_patterns:
            return "まだ学習データが不足しています。最初の試行を行ってください。"
        
        suggestion = f"問題 '{current_problem}' に対する改善提案:\n\n"
        
        if self.learned_lessons:
            suggestion += "⚠️ 避けるべきアプローチ (過去の失敗から):\n"
            for lesson in self.learned_lessons:
                suggestion += f"- {lesson}\n"
            suggestion += "\n"
        
        if self.success_patterns:
            suggestion += "✅ 推奨アプローチ (過去の成功から):\n"
            for pattern in self.success_patterns:
                suggestion += f"- {pattern}\n"
        
        return suggestion

    @kernel_function(description="試行履歴のサマリを取得")
    def get_attempt_summary(self) -> Annotated[str, "試行履歴サマリ"]:
        if not self.attempt_history:
            return "試行履歴はありません"
        
        total = len(self.attempt_history)
        successful = sum(1 for a in self.attempt_history if a["success"])
        failed = total - successful
        
        summary = f"📊 試行履歴サマリ:\n"
        summary += f"- 総試行数: {total}\n"
        summary += f"- 成功: {successful}\n"
        summary += f"- 失敗: {failed}\n"
        summary += f"- 成功率: {(successful/total*100):.1f}%\n\n"
        
        summary += "📋 最新の試行:\n"
        for attempt in self.attempt_history[-3:]:
            status = "✅" if attempt["success"] else "❌"
            summary += f"{status} {attempt['id']}: {attempt['approach']}\n"
        
        return summary

In [6]:
# Reflexionパターンのデモンストレーション
# このセルで学べること: 試行の記録→失敗からの教訓抽出／成功要因の記録→改善提案までの継続的改善サイクル。
# 次セルの比較分析にある「長期運用・学習」の観点を、実行ログで具体的に確認できます。

async def demonstrate_reflexion_pattern():
    """Reflexionパターンのデモンストレーション"""
    
    # 環境変数の読み込み
    load_dotenv()
    
    # OpenAIクライアントの設定
    client = AsyncOpenAI(
        api_key=os.environ.get("GITHUB_TOKEN"), 
        base_url="https://models.inference.ai.azure.com/",
    )
    
    chat_completion_service = OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini",
        async_client=client,
    )
    
    # Reflexionエンジンの初期化
    reflexion_engine = ReflexionEngine()
    
    # Reflexionエージェントの作成
    reflexion_agent = ChatCompletionAgent(
        service=chat_completion_service,
        plugins=[reflexion_engine],
        name="ReflexionAgent",
        instructions="""
        あなたはReflexion（反省）パターンを使用する学習型AIエージェントです。
        
        問題解決のプロセス:
        1. record_attemptで試行を記録
        2. 失敗した場合: reflect_on_failureで原因分析と教訓抽出
        3. 成功した場合: record_success_patternで成功要因を記録
        4. get_learned_knowledgeで過去の学習を確認
        5. suggest_improved_approachで改善されたアプローチを提案
        6. 学習に基づいて次の試行を実施
        
        常に過去の経験から学び、段階的に改善していくことを心がけてください。
        失敗を恐れず、学習の機会として活用してください。
        """,
    )
    
    print("🧠 Reflexion（反省）パターンデモンストレーション")
    print("=" * 50)
    
    # 段階的な学習プロセスのシミュレーション
    learning_scenarios = [
        "複雑な数学問題を解く最適な方法を学習してください。最初は基本的なアプローチから始めて、失敗を通じて改善していきます。",
        "前回の学習を活かして、より効率的なアプローチで同様の問題に取り組んでください。",
        "これまでの学習成果を総合して、最適化された解法を提案してください。"
    ]
    
    for i, scenario in enumerate(learning_scenarios, 1):
        print(f"\n🔄 学習フェーズ {i}")
        print("-" * 40)
        print(f"📝 シナリオ: {scenario}\n")
        
        thread = None
        
        async for response in reflexion_agent.invoke_stream(messages=scenario, thread=thread):
            print(response, end="", flush=True)
            thread = response.thread
        
        print(f"\n\n✅ 学習フェーズ {i} 完了")
        
        # クリーンアップ
        if thread:
            await thread.delete()
        
        # 次のフェーズまで少し待機
        if i < len(learning_scenarios):
            await asyncio.sleep(1)
    
    print("\n" + "=" * 50)
    print("✅ Reflexionパターン学習プロセス完了")

await demonstrate_reflexion_pattern()

🧠 Reflexion（反省）パターンデモンストレーション

🔄 学習フェーズ 1
----------------------------------------
📝 シナリオ: 複雑な数学問題を解く最適な方法を学習してください。最初は基本的なアプローチから始めて、失敗を通じて改善していきます。

📝 試行記録: 1 - 失敗
   アプローチ: 基本的な数式と公式を使って問題を解く
   結果: 問題が解けなかった
📝 試行記録: 1 - 失敗
   アプローチ: 基本的な数式と公式を使って問題を解く
   結果: 問題が解けなかった
🧠 教訓を学習: 失敗パターン: 基本的な数式と公式を使って問題を解く → 原因: 公式の理解が不十分で、適切な数式を選べなかったため問題が解けなかった。
🧠 教訓を学習: 失敗パターン: 基本的な数式と公式を使って問題を解く → 原因: 公式の理解が不十分で、適切な数式を選べなかったため問題が解けなかった。
📝 試行記録: 2 - 成功
   アプローチ: 基本的な公式を例題で学んでから、同様の問題に挑戦する。
   結果: 問題が解けた
📝 試行記録: 2 - 成功
   アプローチ: 基本的な公式を例題で学んでから、同様の問題に挑戦する。
   結果: 問題が解けた
🌟 成功パターンを記録: 成功パターン: 基本的な公式を例題で学んでから、同様の問題に挑戦する。 → 要因: 公式の理解が深まり、例題を使って練習したことで、問題を解くスキルが向上した。


✅ 学習フェーズ 1 完了
🌟 成功パターンを記録: 成功パターン: 基本的な公式を例題で学んでから、同様の問題に挑戦する。 → 要因: 公式の理解が深まり、例題を使って練習したことで、問題を解くスキルが向上した。


✅ 学習フェーズ 1 完了

🔄 学習フェーズ 2
----------------------------------------
📝 シナリオ: 前回の学習を活かして、より効率的なアプローチで同様の問題に取り組んでください。


🔄 学習フェーズ 2
----------------------------------------
📝 シナリオ: 前回の学習を活かして、より効率的なアプローチで同様の問題に取り組んでください。

📝 試行記録: 3 - 成

## 🎯 Agenticデザインパターンの比較分析（このノートブック版）

### 🧩 このノートブックでの実装ポイント
- ReAct: 段階ごとのストリーミング実行＋各フェーズでスレッド破棄（出力肥大・履歴膨張を回避）
- Planning: ツール呼び出し（create_plan / get_next_step / complete_step）を簡潔ログで可視化
- Multi-Agent: CommunicationHubのブロードキャスト対応・複数宛先送信・受信内容をタスクへ埋め込み、確実なハンドオフを実現
- Reflexion: 試行→反省/成功→知識化→改善提案のループを実行ログで確認可能

### パターン別適用場面

| パターン | 適用場面 | メリット | デメリット | 実装コスト |
|---------|----------|----------|-----------|-------------|
| **ReAct** | 試行錯誤が必要な問題 | 透明性高い | 効率が劣る場合あり | 低 |
| **Planning** | 複雑で段階的なタスク | 体系的実行 | 計画変更が困難 | 中 |
| **Multi-Agent** | 専門分野の協調作業 | 高い専門性 | 調整コスト高 | 高 |
| **Reflexion** | 継続的改善が必要 | 学習能力 | 初期成果遅い | 中 |

### 🚀 実用的な組み合わせパターン

**1. ReAct + Reflexion**
```python
# 推論→行動→観察→反省→学習のサイクル
# 問題解決能力が段階的に向上
```

**2. Planning + Multi-Agent**
```python
# 複雑なプランを複数エージェントで分担実行
# 企業プロジェクトに最適
```

**3. Multi-Agent + Reflexion**
```python
# チーム全体での学習と改善
# 組織的知識蓄積
```

### 💡 選択指針

- 問題の性質で選択:
  - 探索的問題 → ReAct
  - 構造化された問題 → Planning  
  - 専門性が必要 → Multi-Agent
  - 繰り返し改善 → Reflexion
- リソースと要件で選択:
  - 開発速度重視 → ReAct
  - 品質重視 → Planning
  - スケール重視 → Multi-Agent
  - 長期運用重視 → Reflexion

### 🎓 学習のまとめ

- 4つの主要Agenticパターンの理解と実装
- パターンの組み合わせによる相乗効果
- 問題に応じた適切なパターン選択
- 実用的なアーキテクチャ設計能力

### ? 次に試す拡張（このノートブックを基に）
- 出力上限・詳細度のトグル（compact/verbose）と各フェーズの表示上限調整
- Multi-Agentの観測性強化（受信箱の件数・プレビューをログ表示）
- リトライ／タイムアウト方針の明示化、失敗時のフォールバック設計
- Reflexionの知識持続化（外部ストア）と再利用プロンプト

### 🏗️ アーキテクチャ設計のガイドライン
- 単一責任原則: 各パターンの役割を明確化
- 疎結合: パターン間の依存を最小化
- 拡張性: 新しいパターンの追加容易性
- テスタビリティ: 各パターンの個別テスト可能性